# How to run

Load models.
Make prediction from sum of probabilities of the models.

Note that the last layer has a softmax activation function, so the output is a probability distribution.
We sum the probabilities from different models and the argmax of the sum will be our prediction

In [2]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
# !unzip images.zip
# !unzip nn_model1.zip
# !unzip nn_model2.zip
# !unzip nn_model3.zip
# !unzip nn_model4.zip
# !unzip nn_model5.zip


In [4]:
import pandas as pd
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

## Load and preproc test set

In [5]:
# Need to use training csv to get embeddings for categorical data
df = pd.read_csv("train.csv", delimiter=',')


def load_image(id):
    img = Image.open('noisy-images/{id}.jpg'.format(id=id))
    data = np.asarray(img)
    return data/255

def get_categorical_embedding(categorical_input, silent=False):
    possible_values = df[categorical_input].unique().tolist()
    num_possible_values = len(possible_values)

    index = lambda x: possible_values.index(x)
    def encoder(x: int):
        encoding = np.zeros(num_possible_values)
        encoding[x] = 1
        return np.ndarray.copy(encoding)

    # index = layers.StringLookup(vocabulary = possible_values, output_mode = 'int')
    # encoder = layers.CategoryEncoding(num_tokens=num_possible_values + 1, output_mode='one_hot')

    
    if not silent:
        print("{n} possible values for {name}".format(n = num_possible_values, name = categorical_input))
        print(possible_values)
    
    return lambda feature: encoder(index(feature))


# for categorical_input in ['category', 'gender', 'baseColour', 'season', 'usage']:
#     possible_values = data_points[categorical_input].unique()
#     num_possible_values = len(possible_values)

#     print("{n} possible values for {name}".format(n = num_possible_values, name = categorical_input))
#     print(possible_values)
    
gender_embedding = get_categorical_embedding('gender')
baseColour_embedding = get_categorical_embedding('baseColour')
season_embedding = get_categorical_embedding('season')
usage_embedding = get_categorical_embedding('usage')


def index_to_category():
    '''
    returns category from index of most likely prediction
    '''
    possible_values = df['category'].unique()
    
    return lambda x: possible_values[x]


embedding_to_category = index_to_category()

del df

5 possible values for gender
['Men', 'Women', 'Girls', 'Unisex', 'Boys']
46 possible values for baseColour
['Tan', 'Blue', 'White', 'Black', 'Beige', 'Pink', 'Green', 'Red', 'Brown', 'Grey', 'Yellow', 'Magenta', 'Steel', 'Purple', 'Orange', 'Silver', 'Navy Blue', 'Maroon', 'Gold', 'Olive', 'Cream', 'Peach', 'Lavender', 'Coffee Brown', 'Grey Melange', 'Teal', 'Rust', 'Multi', 'Charcoal', 'Turquoise Blue', 'Rose', 'Off White', 'Skin', 'Khaki', 'Metallic', 'Nude', 'Mustard', 'Copper', 'Burgundy', 'Sea Green', 'Mauve', 'Mushroom Brown', 'Bronze', 'Taupe', 'Lime Green', 'Fluorescent Green']
4 possible values for season
['Summer', 'Fall', 'Winter', 'Spring']
7 possible values for usage
['Casual', 'Sports', 'Formal', 'Ethnic', 'Smart Casual', 'Party', 'Travel']


In [6]:
df = pd.read_csv("test_preproc.csv", delimiter=',')

df['noisyTextDescription'] = df['noisyTextDescription'].map(lambda x: np.array(x[1:-1].split(), dtype=float))
df["noisyImage"] = df.apply(lambda x: load_image(x['id']), axis=1)

df['gender'] = df['gender'].map(lambda x: gender_embedding(x))
df['baseColour'] = df['baseColour'].map(lambda x: baseColour_embedding(x))
df['season'] = df['season'].map(lambda x: season_embedding(x))
df['usage'] = df['usage'].map(lambda x: usage_embedding(x))

df["categoricalData"] = df.apply(lambda x: np.concatenate((x['gender'], x['baseColour'], x['season'], x['usage']), axis=0), axis=1)

x_text_test = df['noisyTextDescription'].to_numpy()
x_img_test = df['noisyImage'].to_numpy()
x_categorical_test = df['categoricalData'].to_numpy()

x_img_test = np.vstack([np.array([img]) for img in x_img_test])
x_text_test = np.vstack([np.array([text]) for text in x_text_test])
x_categorical_test = np.vstack([np.array([categorical]) for categorical in x_categorical_test])

test_ids = df['id'].to_numpy()

In [7]:
print(test_ids)
print(type(test_ids))

[55581  8832 29112 ... 41829 27416 24079]
<class 'numpy.ndarray'>


## Load models

In [8]:
current_predictions = np.zeros(shape=(len(test_ids), 27))

for i in range(1, 6):
    model_name = 'nn_model{n}'.format(n=i)
    model = tf.keras.models.load_model(model_name)
    prediction = model.predict((x_categorical_test, x_text_test, x_img_test), verbose=2)
    print(type(prediction))
    print(prediction.shape)
    current_predictions = current_predictions + prediction


676/676 - 15s - 15s/epoch - 23ms/step
<class 'numpy.ndarray'>
(21628, 27)
676/676 - 7s - 7s/epoch - 10ms/step
<class 'numpy.ndarray'>
(21628, 27)
676/676 - 6s - 6s/epoch - 9ms/step
<class 'numpy.ndarray'>
(21628, 27)
676/676 - 4s - 4s/epoch - 5ms/step
<class 'numpy.ndarray'>
(21628, 27)
676/676 - 8s - 8s/epoch - 12ms/step
<class 'numpy.ndarray'>
(21628, 27)


In [18]:
for id, prediction in zip(test_ids, current_predictions):
    print('{} , {}'.format(id, embedding_to_category(np.argmax(prediction))))

Streaming output truncated to the last 5000 lines.
54587 , Shoes
35704 , Topwear
20840 , Topwear
5781 , Innerwear
10994 , Belts
57111 , Shoes
27485 , Topwear
6589 , Eyewear
13451 , Flip Flops
17022 , Shoes
12649 , Topwear
18011 , Innerwear
57203 , Bottomwear
37309 , Headwear
31430 , Eyewear
38191 , Wallets
4269 , Shoes
10681 , Watches
32043 , Topwear
40070 , Lips
43220 , Shoes
16422 , Flip Flops
53783 , Sandal
20360 , Topwear
12213 , Flip Flops
43054 , Topwear
16246 , Eyewear
29514 , Watches
52304 , Topwear
5212 , Topwear
50329 , Fragrance
54055 , Bags
17303 , Topwear
35216 , Shoes
23740 , Shoes
6303 , Topwear
27026 , Ties
22822 , Belts
40598 , Fragrance
39137 , Topwear
19478 , Wallets
35927 , Fragrance
43949 , Topwear
49018 , Topwear
22850 , Topwear
7211 , Topwear
36038 , Jewellery
37725 , Watches
6801 , Bottomwear
9879 , Bottomwear
35710 , Wallets
44648 , Watches
4666 , Topwear
58180 , Topwear
51253 , Topwear
32348 , Bags
27099 , Topwear
32174 , Watches
33028 , Topwear
25069 , Innerw

In [19]:
predictions = [embedding_to_category(np.argmax(prediction)) for prediction in current_predictions]

dict = {'id': test_ids, 'category': predictions}

df = pd.DataFrame(dict) 

df.to_csv('submission.csv', index=False) 

In [20]:
! tail submission.csv

46867,Topwear
13362,Shoes
39733,Topwear
55657,Topwear
34995,Topwear
17255,Topwear
39606,Innerwear
41829,Wallets
27416,Topwear
24079,Innerwear
